In [1]:
import re
from copy import deepcopy
from os import path as osp
import open3d as o3d
from open3d.web_visualizer import draw
import sys
import os

import numpy as np
import torch

lidar_path ='/home/server-003/workspace/data/superb_mobis/jupyter_apps/asset/005be846-3ada-4f4c-985e-5fbc0b136b2f/pointclouds_00000001.bin'
calib_path ='/home/server-003/dl/mmdetection3d/data/superb/training/calib/000000.txt'
label_path ='/home/server-003/dl/mmdetection3d/data/superb/training/label_2/000000.txt'

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


In [2]:
def depth_color(val, min_d=0, max_d=120):
    """ 
    print Color(HSV's H value) corresponding to distance(m) 
    close distance = red , far distance = blue
    """
    np.clip(val, 0, max_d, out=val)  # max distance is 120m but usually not usual
    return (((val - min_d) / (max_d - min_d)) * 120).astype(np.uint8)

In [3]:
def load_bin(bin_path):
    points = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)
    return points[:, :3]

In [4]:
def get_calib_param(filepath):
    """ 
    get Rotation(R : 3x3), Translation(T : 3x1) matrix info 
    using R,T matrix, we can convert velodyne coordinates to camera coordinates
    """
    with open(filepath, "r") as f:
        file = f.readlines()

        for line in file:
            (key, val) = line.split(':', 1)
            if key == 'Tr_velo_to_cam':
                RT = np.fromstring(val, sep=' ')
                RT = RT.reshape(3, 4)
        
    return RT


In [5]:
def get_label_param(filepath):
    """ 
    get Rotation(R : 3x3), Translation(T : 3x1) matrix info 
    using R,T matrix, we can convert velodyne coordinates to camera coordinates
    """
    with open(filepath, "r") as f:
        file = f.readlines()
        values=[]
        label_id=[]
        for line in file:
            strings = line.split(' ')
            value= np.asarray(np.array(strings[1:]), np.float32).reshape(1,-1)
            label_id.append(strings[0])
            values.append(value)
            
    return np.array(label_id),np.array(values)


In [6]:
label_id,values=get_label_param(label_path)

In [16]:

velo_points = load_bin(lidar_path)
x = velo_points[:, 0]
y = velo_points[:, 1]
z = velo_points[:, 2]
dist = np.sqrt(x**2+y**2+z**2)
ones = np.ones([velo_points.shape[0],2]).transpose()
ones = ones*255
color=depth_color(dist,0,70)
color=np.ceil(np.array([color]))
colors = np.concatenate((color,ones),axis=0)



In [17]:
extrinsic_param = get_calib_param(calib_path)
homo=np.array([0,0,0,1]).reshape(1,4)
extrinsic_param_homo = np.concatenate([extrinsic_param,homo],axis=0)
cam_to_lidar = np.linalg.inv(extrinsic_param_homo)


In [18]:
center = values[:,:,10:13].squeeze()
one=np.ones(center.shape[0]).reshape(4,1)
center_cam_homo = np.concatenate([center,one],axis=1)

size = values[:,:,7:10].squeeze()
rot = values[:,:,13].squeeze().reshape(-1,1)

center_lidar = np.matmul(cam_to_lidar,center_cam_homo.transpose())
center_lidar = center_lidar.transpose()
center_lidar = center_lidar[:,:3]

dim=np.empty(size.shape)

dim[:,0]=size[:,1]
dim[:,1]=size[:,2]
dim[:,2]=size[:,0]

center_lidar[:,2] =  center_lidar[:,2] +(size[:,2] / 2)



yaw = np.zeros([center.shape[0],3])
yaw[:,2]=rot[:,0]


draw_box = []
cloud = o3d.geometry.PointCloud()
cloud.points = o3d.utility.Vector3dVector(velo_points[:,:3])
cloud.colors = o3d.utility.Vector3dVector(colors.transpose())
draw_box.append(cloud)

for i in range(center.shape[0]):
    rot_mat = o3d.geometry.get_rotation_matrix_from_xyz(yaw[i])
    box3d = o3d.geometry.OrientedBoundingBox(center[i], rot_mat, dim[i])
    draw_box.append(box3d)


draw(draw_box)


[Open3D INFO] Window window_0 created.
[Open3D INFO] EGL headless mode enabled.
[Open3D INFO] ICE servers: {"stun:stun.l.google.com:19302", "turn:user:password@34.69.27.100:3478", "turn:user:password@34.69.27.100:3478?transport=tcp"}
FEngine (64 bits) created at 0x7fc988008b20 (threading is enabled)
[Open3D INFO] Set WEBRTC_STUN_SERVER environment variable add a customized WebRTC STUN server.
[Open3D INFO] WebRTC Jupyter handshake mode enabled.
EGL(1.5)
OpenGL(4.6)
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]


../src/intel/isl/isl.c:2105: FINISHME: ../src/intel/isl/isl.c:isl_surf_supports_ccs: CCS for 3D textures is disabled, but a workaround is available.


WebVisualizer(window_uid='window_0')

[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Called HTTP API (custom handshake)] /api/call
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceCandidate
[Open3D INFO] DataChannelObserver::OnStateChange label: ServerDataChannel, state: open, peerid: 0.6289977270313087
[Open3D INFO] DataChannelObserver::OnStateChange label: ClientDataChannel, state: open, peerid: 0.6289977270313087
[Open3D INFO] Sending init frames to window_0.
